In [30]:
import mysql.connector
import pandas as pd

# Informations de connexion à la base de données RDS
endpoint = 'database-datacoders.c3m48simgdzn.eu-west-3.rds.amazonaws.com'
username = 'admin'
password = 'dataCoders2024+'
database_name = 'db_datacoders'
port = 3306  # Le port par défaut pour MySQL est 3306

# Création de la connexion
conn = mysql.connector.connect(
    host=endpoint,
    user=username,
    passwd=password,
    database=database_name,
    port=port
)

# Création d'un curseur
cursor = conn.cursor()

# Exemple d'exécution d'une requête SQL
cursor.execute('''
               SELECT * 
               FROM user_info as ui
               LEFT JOIN login_user as lu USING (id_login_user)
               LEFT JOIN mentor_sujet as ms ON (lu.id_login_user = ms.id_mentor)
               LEFT JOIN mentorat as mt ON (mt.id_mentorat = ms.id_mentorat)
               LEFT JOIN sujet as sj ON (lu.id_login_user = sj.id_utilisateur_mentoret)
               LEFT JOIN calendrier as c ON (lu.id_login_user = c.id_utilisateur_mentoret)
               ''')
rows = cursor.fetchall()

# # Affichage des résultats
# for row in rows[:20]:
#     print(row)



column_names = [desc[0]+str(i) for desc, i in zip(cursor.description, range(len(cursor.description)))]
print(column_names)

# Création du DataFrame pandas avec noms de colonnes préfixés par les noms des tables
# column_names_prefixed = [f"{table}_{column}" if table else column for table, column in zip(table_names, column_names)]
df = pd.DataFrame(rows, columns=column_names)

# Fermeture du curseur et de la connexion
cursor.close()
conn.close()


# enregistrer au format parquet
df.to_parquet('user_info.parquet', index=False)

['id_login_user0', 'id_user_info1', 'prenom2', 'nom3', 'sexe4', 'age5', 'mentor6', 'lgbt7', 'handicap8', 'type_handicap9', 'niveau_professionnel10', 'categorie_socio_professionel11', 'nombre_de_participation12', 'id_mentorat13', 'email14', 'mot_de_passe15', 'created_at16', 'last_connection_at17', 'id_mentor18', 'id_mentorat19', 'id_mentorat20', 'type_mentorat21', 'id_sujet22', 'id_mentorat23', 'id_utilisateur_mentor24', 'id_utilisateur_mentoret25', 'sujet_du_mentorat26', 'created_at27', 'updated_at28', 'id_utilisateur_mentor29', 'id_utilisateur_mentoret30', 'id_mentorat31', 'created_at32', 'updated_at33']


In [31]:
df = pd.read_parquet(r'C:\Users\dimle\Documents\clone_repo\HackathonWCS\ML\user_info.parquet')

df

,id_login_user0,id_user_info1,prenom2,nom3,sexe4,age5,mentor6,lgbt7,handicap8,type_handicap9,...,id_utilisateur_mentor24,id_utilisateur_mentoret25,sujet_du_mentorat26,created_at27,updated_at28,id_utilisateur_mentor29,id_utilisateur_mentoret30,id_mentorat31,created_at32,updated_at33
0,789,789,Mary,Martin,Agender,24,1,Yes,0,None,...,NaN,NaN,None,NaT,NaT,NaN,NaN,NaN,NaT,NaT
1,789,789,Mary,Martin,Agender,24,1,Yes,0,None,...,NaN,NaN,None,NaT,NaT,NaN,NaN,NaN,NaT,NaT
2,790,790,William,Jones,Agender,46,1,Yes,0,None,...,NaN,NaN,None,NaT,NaT,NaN,NaN,NaN,NaT,NaT
3,790,790,William,Jones,Agender,46,1,Yes,0,None,...,NaN,NaN,None,NaT,NaT,NaN,NaN,NaN,NaT,NaT
4,791,791,Ricky,Barnes,Intersex,53,0,Yes,0,None,...,NaN,NaN,None,NaT,NaT,923.0,791.0,1.0,2024-07-04 10:05:57,2024-07-04 10:05:57
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
1538,1785,1785,Monica,Meyers,Transgender Male,27,1,No,1,Auditif,...,NaN,NaN,None,NaT,NaT,NaN,NaN,NaN,NaT,NaT
1539,1786,1786,Ruth,Ford,Agender,40,0,No,0,None,...,NaN,NaN,None,NaT,NaT,1516.0,1786.0,3.0,2024-07-04 10:05:57,2024-07-04 10:05:57
1540,1787,1787,Courtney,Shah,Genderqueer,62,1,Yes,0,None,...,NaN,NaN,None,NaT,NaT,NaN,NaN,NaN,NaT,NaT
1541,1787,1787,Courtney,Shah,Genderqueer,62,1,Yes,0,None,...,NaN,NaN,None,NaT,NaT,NaN,NaN,NaN,NaT,NaT
